### 1NN против RandomForest

Необходимо сравнить долю ошибок классификаторов 1NN и RandomForest на датасете digits

In [1]:
from sklearn.datasets import load_digits
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
import numpy as np
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Будем работать с датасетом digits

In [3]:
digits = load_digits()

Выборку необходимо разделить таким образом, чтобы последние 25% объектов для контроля качества

In [7]:
def non_shuffling_train_test_split(X, y, test_size=0.25):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = non_shuffling_train_test_split(digits.data, digits.target, test_size=0.25)

In [10]:
classifier1 = KNeighborsClassifier(n_neighbors=1)  
classifier1.fit(X_train, y_train)  
y_pred1 = classifier1.predict(X_test) 
print(1- accuracy_score(y_test, y_pred1))

0.03786191536748329


In [11]:
classifier2 = RandomForestClassifier(n_estimators=1000)
classifier2.fit(X_train, y_train)
y_pred2 = classifier2.predict(X_test) 
print(1- accuracy_score(y_test, y_pred2))

0.06458797327394206


Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

### Бэггинг и случайный лес

In [14]:
X = digits.data
y = digits.target 

1. DecisionTreeClassifier с настройками по умолчанию 

In [15]:
clf = DecisionTreeClassifier()
scores = cross_val_score(clf, X, y, cv=10)
scores.mean()

0.831390982664509

2. BaggingClassifier c настройками по умолчанию и числом деревьев = 100

In [5]:
clf = BaggingClassifier(n_estimators=100)
scores = cross_val_score(clf, digits.data, digits.target, cv=10)
scores.mean()

0.9271833410531739

Обучение не на всех d признаках, а на sqrt(d) признаках

In [6]:
clf = BaggingClassifier(n_estimators=100, max_features=int(np.sqrt(digits.data.shape[1])))
scores = cross_val_score(clf, digits.data, digits.target, cv=10)
scores.mean()

0.9303579474763959

Отбор признаков при построении каждой вершины дерева (бэггинг на рандомизированных деревьях)

In [16]:
import numpy as np
clf = BaggingClassifier(base_estimator = DecisionTreeClassifier(max_features=int(np.sqrt(digits.data.shape[1]))),n_estimators=100,random_state = 1)
scores = cross_val_score(clf, digits.data, digits.target, cv=10)
scores.mean()

0.9500349636021452